In [39]:
%load_ext autoreload
%autoreload 2

from nlp_utils import PreProcessing
from nlp_utils import TopicModeling
from google.cloud import bigquery
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
nome_da_aba = "Base 34K"
df = pd.read_excel('data/data.xlsx', sheet_name=nome_da_aba)[["video_id","channel_id","video_title", "Label"]]

novo_nome_colunas = [coluna.replace(' ', '_').lower() for coluna in df.columns]
df['video_title'] = df['video_title'].astype(str)
df.columns = novo_nome_colunas

df.head()

,video_id,channel_id,video_title,label
0,-GjJTsTHL_U,UC-8Uff7i2h5qtIvjXJDJqYA,"Arranjos Florais: 19/08, na Fnac da Paulista (SP)",Gardening
1,-RM0Em8oSd8,UC-8Uff7i2h5qtIvjXJDJqYA,Saiba tirar muda da orquídea Phalaenopsis (kei...,Gardening
2,0ORo9kvtOrE,UC-8Uff7i2h5qtIvjXJDJqYA,Regador com garrafa PET e outros truques de ja...,Gardening
3,10L1DZTP1Ak,UC-8Uff7i2h5qtIvjXJDJqYA,Curso de Agricultura Biodinâmica (Fazendeira p...,Gardening
4,1dnkMo7vlBY,UC-8Uff7i2h5qtIvjXJDJqYA,11 plantas perfeitas para ter em local sem sol,Gardening


In [41]:
#NLP
stop_word_list =   ["aqui", "gente", "oi", "então", "ai", "aí", "ufa", "coisa", "[Aplausos]", "[Música]", "né","tá", "vídeo","ne", "ta", "ha", "one", "at",
                    "casar", "coisa","canal", "tipo", "vezes", "mano", "meio", "ea", "eo", "só", "ae", "eis", "caraca", "caramba", "feat", "ft",
                    "graciele_lacerda",  "flavia_calino", "richard_rasmussen", "rezende_evil", "divo_depressao", "receitar_cris", "luisa_mell",
                    "leo_stronda", "divo_depressao", "estevar_mundo", "sal_flor", "tiago_fonseca", "graciele_lacerda", "organizar_frescura", "eduardo_bueno",
                    "debora_aladim", "paula_stephania", "dani_noce	", "tpm_ju", "prof_paulo", "meteoro_doc", "mariano_saad", "familia_brancoala", "osf_rafar",
                    "oliveira", "isabella_fiorentino", "pit_money", "luca_pit", "murilo_coutar", "bianca_andrade", "manual_mundo", "receitar_pai", "dani_noce", "thiago_venturo",
                    "danilo_gentili", "tato_fersoza", "teodoro_tato", "of_the", "like", "it", "house", "danilo", "tatá", "fersoza", "nerdologia", "leo", "stronda", "ana", "flavia", "calina",
                    "thiago", "ventura"
                    ] #lista de stop words extendida

pp = PreProcessing.PreProcessing(stop_words_list=stop_word_list)

16:24:00 - Adicionanado novas stopwords.
16:24:00 -    Serão adcionada(s) 86 palavra(s)
16:24:00 -    Tamanho da lista de stopwords: 489
16:24:00 -    Tamanho da lista de stopwords: 491


In [42]:
start_time = time.time()
# lista_documentos = df_video_transcriptions["transcription_pt"].to_list()
lista_documentos = df["video_title"].to_list()

allowed_postags = ["NOUN","VERB", "PROPN"] # filtro de postags
min_count=5  # Ngram - confiança
threshold=10 # Ngram - suporte
grau=4       # Ngram - quadrigrams
token_to_lower=True #Converter tudo para minusculo 
token_acentuacao=False #Manter Acentuação

corpus, modelo_ngram = pp.pipeline_pre_processamento(lista_documentos, 
                                allowed_postags=allowed_postags,
                                token_to_lower=token_to_lower, 
                                token_acentuacao=token_acentuacao,
                                n_gram_grau=grau,
                                n_gram_min_count=min_count,
                                n_gram_threshold=threshold,
                                stop_word_list=stop_word_list,
                                )
corpus = pp.remover_stop_words(corpus)

print("-> %s Segundos" % (time.time() - start_time))

16:24:01 - Iniciando pipeline de pré processamento:
16:24:01 - Removendo Urls:
16:24:01 - Filtrando Pos Tags
16:26:28 - Tokenizando
16:26:29 - Removendo Stop Words
16:26:29 -     -> Expandindo Stop Words
16:26:29 - Adicionanado novas stopwords.
16:26:29 -    Serão adcionada(s) 86 palavra(s)
16:26:29 -    Tamanho da lista de stopwords: 491
16:26:29 -    Tamanho da lista de stopwords: 491
16:26:29 - Lematizando
16:28:03 - Ngrams: Extraindo os Ngrams a partir de um modelo pré treinado.
16:28:05 - Filtrando tokens menores que 2
-> 245.43864059448242 Segundos


In [43]:
df["corpus"] = corpus
df.head()

,video_id,channel_id,video_title,label,corpus
0,-GjJTsTHL_U,UC-8Uff7i2h5qtIvjXJDJqYA,"Arranjos Florais: 19/08, na Fnac da Paulista (SP)",Gardening,"[arranjo, Fnac, paulista, sp]"
1,-RM0Em8oSd8,UC-8Uff7i2h5qtIvjXJDJqYA,Saiba tirar muda da orquídea Phalaenopsis (kei...,Gardening,"[tirar, mudar, orquidea, phalaenopsis]"
2,0ORo9kvtOrE,UC-8Uff7i2h5qtIvjXJDJqYA,Regador com garrafa PET e outros truques de ja...,Gardening,"[regador, garrafar, pet, truque, jardinagem_ca..."
3,10L1DZTP1Ak,UC-8Uff7i2h5qtIvjXJDJqYA,Curso de Agricultura Biodinâmica (Fazendeira p...,Gardening,"[curso, agricultura, biodinamico, fazendeiro_s..."
4,1dnkMo7vlBY,UC-8Uff7i2h5qtIvjXJDJqYA,11 plantas perfeitas para ter em local sem sol,Gardening,"[planta, sol]"


In [53]:
def create_top_n_words_dataframe(label,freq_dict, n):
    sorted_words = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
    top_n_words = sorted_words[:n]
    df = pd.DataFrame(top_n_words, columns=['palavra', 'frequencia'])
    df["label"] = label
    return df

In [56]:
lista_label = df["label"].unique().tolist()
lista_df_labels = []
for label in lista_label:
    df_label = df[df["label"]==label]
    corpus_label = df_label["corpus"].tolist() 
    id2word_label = pp.montar_id2word(corpus_label)
    bow_corpus_label = pp.montar_bow(corpus_label, id2word_label)

    word_freq = {id2word_label.get(id): count for id, count in id2word_label.dfs.items()}
    lista_df_labels.append(create_top_n_words_dataframe(label,word_freq, 15))


In [57]:
df_resultado = pd.concat(lista_df_labels, axis=0)
df_resultado.head()

,palavra,frequencia,label
0,planta,108,Gardening
1,orquidea,35,Gardening
2,horta,35,Gardening
3,Jardim,28,Gardening
4,dica,27,Gardening


In [58]:
depara_anomizado = {label: f"l_{i}" for i, label in enumerate(lista_label)}
depara_anomizado

{'Gardening': 'l_0',
 'Education': 'l_1',
 'People, Behavior and Lifestyle': 'l_2',
 'Gastronomy': 'l_3',
 'Decoration, Organization and DIY': 'l_4',
 'Family': 'l_5',
 'Health and healthy lifestyle': 'l_6',
 'Travel, learnings and curiosities': 'l_7',
 'Culture and Entertainment': 'l_8',
 'Sports': 'l_9',
 'Gaming': 'l_10',
 'Entertainment/general': 'l_11',
 'Economics, Entrepreneurship and Business': 'l_12',
 'Tech': 'l_13',
 'Fashion/Lifestyle': 'l_14',
 'Pets & Animals ': 'l_15',
 'Beauty': 'l_16',
 'Military': 'l_17',
 'Politics, Economy and News': 'l_18'}

In [59]:
df_label_anomizado = df_resultado.copy()
df_label_anomizado["label"] = df_label_anomizado.apply(lambda x: depara_anomizado[x["label"]],axis=1) 

In [61]:
display(df_resultado.tail())
df_label_anomizado.tail()

,palavra,frequencia,label
10,pt,34,"Politics, Economy and News"
11,Kim,34,"Politics, Economy and News"
12,debate,32,"Politics, Economy and News"
13,golpe,28,"Politics, Economy and News"
14,vivo,28,"Politics, Economy and News"


,palavra,frequencia,label
10,pt,34,l_18
11,Kim,34,l_18
12,debate,32,l_18
13,golpe,28,l_18
14,vivo,28,l_18


In [62]:
df_resultado.to_csv("data\\top_words_labels.csv")
df_label_anomizado.to_csv("data\\top_words_anon.csv")